In [256]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dtale
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import preprocessing
from sklearn.metrics import r2_score
import pickle

%matplotlib
%matplotlib inline
pd.options.display.float_format = '{:.4f}'.format #set it to convert scientific noations such as 4.225108e+11 to 422510842796.00
pd.set_option('display.max_columns', 100) #  display all the columns
pd.set_option('display.max_rows', 100) # display all the rows
np.set_printoptions(suppress=True,formatter={'float_kind':'{:f}'.format})

Using matplotlib backend: MacOSX


# Loading data

In [262]:
df = pd.read_csv('Data/Binary_label_K_bins_train.csv')
#df = df.sample(10000)
df.drop('Unnamed: 0',axis='columns',inplace=True)
X_train = df.drop('training labels',axis='columns')
y_train = df[['training labels']]

In [266]:
#df = pd.read_csv('Data/Binary_label_K_bins_train.csv')
#df = df.sample(1000)
#df.drop('Unnamed: 0',axis='columns',inplace=True)
X_test = df.drop('training labels',axis='columns')
y_test = df[['training labels']]


# Logistic Regression

In [268]:
logisticRegr = LogisticRegression()
logisticRegr.fit(X_train, y_train)

/Users/babyhandzzz/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/babyhandzzz/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [269]:
train_score = logisticRegr.score(X_train, y_train)
test_score = logisticRegr.score(X_test, y_test)

# These results are derived using the entire dataset
* using only those labels that signify a definitive outcome

In [270]:
print('Preliminary results for Logistic Regression:')
print('train score: ',train_score)
print('test score: ',test_score)

Preliminary results for Logistic Regression:
train score:  0.9836297813663861
test score:  0.9836297813663861


# Saving the model

In [221]:
filename = 'Models/log_reg_model.sav'
pickle.dump(logisticRegr, open(filename, 'wb')) 

#load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))

# Learning Curves